In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [29]:
albums=pd.read_csv('albums.csv', delimiter='\t') 
artists=pd.read_csv('artists.csv', delimiter='\t')
releases=pd.read_csv('releases.csv', delimiter='\t')
songs=pd.read_csv('songs.csv', delimiter='\t')
tracks=pd.read_csv('tracks.csv', delimiter='\t')

In [31]:
acoustic_features = pd.read_csv('acoustic_features.csv', delimiter='\t')
lyrics = pd.read_csv('lyrics.csv', delimiter='\t')

In [32]:
print(" The columns types are \n{}".format(albums.dtypes)) 
print("----------------------------")
print("The dimension of the data is {}".format(albums.shape))
print("----------------------------")
print("The count of missing values in each column is \n{}".format(albums.isnull().sum()))
print("----------------------------")
print("The count of duplicates in the data is \n{}".format(albums.duplicated().sum()))
print("------------- ---------------")
albums.head()

 The columns types are 
album_id        object
name            object
billboard       object
artists         object
popularity       int64
total_tracks     int64
album_type      object
image_url       object
dtype: object
----------------------------
The dimension of the data is (26519, 8)
----------------------------
The count of missing values in each column is 
album_id        0
name            0
billboard       0
artists         0
popularity      0
total_tracks    0
album_type      0
image_url       0
dtype: int64
----------------------------
The count of duplicates in the data is 
0
------------- ---------------


,album_id,name,billboard,artists,popularity,total_tracks,album_type,image_url
0,5n1GSzC1Reao29ScnpLYqp,Dying To Live,Dying To Live,{'46SHBwWsqBkxI7EeeBEQG7': 'Kodak Black'},83,16,album,https://i.scdn.co/image/db2133234d458f432ca207...
1,6UYZEYjpN1DYRW0kqFy9ZE,Championships,Championships,{'20sxb77xiYeusSH8cVdatc': 'Meek Mill'},85,19,album,https://i.scdn.co/image/77eb7c17cafe5503c58661...
2,7uVimUILdzSZG4KKKWToq0,Christmas (Deluxe Special Edition),Christmas,{'1GxkXlMwML1oSg5eLPiAz3': 'Michael Bublé'},60,20,album,https://i.scdn.co/image/2d6ee8d4fb5a45abf35cd3...
3,35s58BRTGAEWztPo9WqCIs,Spider-Man: Into the Spider-Verse (Soundtrack ...,Spider-Man: Into The Spider-Verse,{'0LyfQWJT6nXafLPZqxe9Of': 'Various Artists'},92,13,compilation,https://i.scdn.co/image/3aa37254a41cf96e815725...
4,41GuZcammIkupMPKH2OJ6I,ASTROWORLD,ASTROWORLD,{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},91,17,album,https://i.scdn.co/image/cdca7dc20c778ada42fb18...


In [33]:
print(" The columns types are \n{}".format(artists.dtypes)) 
print("----------------------------")
print("The dimension of the data is {}".format(artists.shape))
print("----------------------------")
print("The count of missing values in each column is \n{}".format(artists.isnull().sum()))
print("----------------------------")
print("The count of duplicates in the data is \n{}".format(artists.duplicated().sum()))
print("------------- ---------------")
artists.head()

 The columns types are 
artist_id       object
name            object
followers      float64
popularity       int64
artist_type     object
main_genre      object
genres          object
image_url       object
dtype: object
----------------------------
The dimension of the data is (11518, 8)
----------------------------
The count of missing values in each column is 
artist_id      0
name           0
followers      2
popularity     0
artist_type    0
main_genre     0
genres         0
image_url      0
dtype: int64
----------------------------
The count of duplicates in the data is 
0
------------- ---------------


,artist_id,name,followers,popularity,artist_type,main_genre,genres,image_url
0,66CXWjxzNUsdJxJ2JdwvnR,Ariana Grande,34554242.0,96,singer,dance pop,"['dance pop', 'pop', 'post-teen pop']",https://i.scdn.co/image/b1dfbe843b0b9f54ab2e58...
1,26VFTg2z8YR0cCuwLzESi2,Halsey,7368242.0,90,singer,dance pop,"['dance pop', 'electropop', 'etherpop', 'indie...",https://i.scdn.co/image/22a5f3d8c42bc7cb55215e...
2,0Y5tJX1MQlPlqiwlOH1tJY,Travis Scott,6313709.0,94,rapper,pop,"['pop', 'pop rap', 'rap']",https://i.scdn.co/image/dc5eba5e032c2e5bc4d42c...
3,246dkjvS1zLTtiykXe5h60,Post Malone,16737002.0,96,rapper,dfw rap,"['dfw rap', 'pop', 'rap']",https://i.scdn.co/image/f9d8b742b66609f12da023...
4,1zNqQNIdeOUZHb8zbZRFMX,Swae Lee,483032.0,89,singer,trap music,['trap music'],https://i.scdn.co/image/a177469870b41f7e17e3b5...


In [34]:
print(" The columns types are \n{}".format(releases.dtypes)) 
print("----------------------------")
print("The dimension of the data is {}".format(releases.shape))
print("----------------------------")
print("The count of missing values in each column is \n{}".format(releases.isnull().sum()))
print("----------------------------")
print("The count of duplicates in the data is \n{}".format(releases.duplicated().sum()))
print("------------- ---------------")
releases.head()

 The columns types are 
artist_id                 object
album_id                  object
release_date              object
release_date_precision    object
dtype: object
----------------------------
The dimension of the data is (26522, 4)
----------------------------
The count of missing values in each column is 
artist_id                 0
album_id                  0
release_date              0
release_date_precision    0
dtype: int64
----------------------------
The count of duplicates in the data is 
0
------------- ---------------


,artist_id,album_id,release_date,release_date_precision
0,46SHBwWsqBkxI7EeeBEQG7,5n1GSzC1Reao29ScnpLYqp,2018-12-14,day
1,20sxb77xiYeusSH8cVdatc,6UYZEYjpN1DYRW0kqFy9ZE,2018-11-30,day
2,1GxkXlMwML1oSg5eLPiAz3,7uVimUILdzSZG4KKKWToq0,2012-11-09,day
3,0LyfQWJT6nXafLPZqxe9Of,35s58BRTGAEWztPo9WqCIs,2018-12-14,day
4,0Y5tJX1MQlPlqiwlOH1tJY,41GuZcammIkupMPKH2OJ6I,2018-08-03,day


In [35]:
print(" The columns types are \n{}".format(songs.dtypes)) 
print("----------------------------")
print("The dimension of the data is {}".format(songs.shape))
print("----------------------------")
print("The count of missing values in each column is \n{}".format(songs.isnull().sum()))
print("----------------------------")
print("The count of duplicates in the data is \n{}".format(songs.duplicated().sum()))
print("------------- ---------------")
songs.head()

 The columns types are 
song_id       object
song_name     object
billboard     object
artists       object
popularity     int64
explicit        bool
song_type     object
dtype: object
----------------------------
The dimension of the data is (20405, 7)
----------------------------
The count of missing values in each column is 
song_id       0
song_name     0
billboard     0
artists       0
popularity    0
explicit      0
song_type     0
dtype: int64
----------------------------
The count of duplicates in the data is 
0
------------- ---------------


,song_id,song_name,billboard,artists,popularity,explicit,song_type
0,3e9HZxeyfWwjeyPAMmWSSQ,"thank u, next","('Thank U, Next', 'Ariana Grande')",{'66CXWjxzNUsdJxJ2JdwvnR': 'Ariana Grande'},86,True,Solo
1,5p7ujcrUXASCNwRaWNHR1C,Without Me,"('Without Me', 'Halsey')",{'26VFTg2z8YR0cCuwLzESi2': 'Halsey'},87,True,Solo
2,2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,"('Sicko Mode', 'Travis Scott')",{'0Y5tJX1MQlPlqiwlOH1tJY': 'Travis Scott'},85,True,Solo
3,3KkXRkHbMCARz0aVfEt68P,Sunflower - Spider-Man: Into the Spider-Verse,('Sunflower (Spider-Man: Into The Spider-Verse...,"{'246dkjvS1zLTtiykXe5h60': 'Post Malone', '1zN...",92,False,Collaboration
4,1rqqCSm0Qe4I9rUvWncaom,High Hopes,"('High Hopes', 'Panic! At The Disco')",{'20JZFwl6HVl6yg8a4H3ZqK': 'Panic! At The Disco'},86,False,Solo


In [36]:
print(" The columns types are \n{}".format(tracks.dtypes)) 
print("----------------------------")
print("The dimension of the data is {}".format(tracks.shape))
print("----------------------------")
print("The count of missing values in each column is \n{}".format(tracks.isnull().sum()))
print("----------------------------")
print("The count of duplicates in the data is \n{}".format(tracks.duplicated().sum()))
print("------------- ---------------")
tracks.head()

 The columns types are 
song_id                   object
album_id                  object
track_number               int64
release_date              object
release_date_precision    object
dtype: object
----------------------------
The dimension of the data is (20405, 5)
----------------------------
The count of missing values in each column is 
song_id                   0
album_id                  0
track_number              0
release_date              0
release_date_precision    0
dtype: int64
----------------------------
The count of duplicates in the data is 
0
------------- ---------------


,song_id,album_id,track_number,release_date,release_date_precision
0,3e9HZxeyfWwjeyPAMmWSSQ,2fYhqwDWXjbpjaIJPEfKFw,11,2019-02-08,day
1,5p7ujcrUXASCNwRaWNHR1C,0zzrCTzvL4ZmR42xF46Afm,1,2018-10-04,day
2,2xLMifQCjDGFmkHkpNLD9h,41GuZcammIkupMPKH2OJ6I,3,2018-08-03,day
3,3KkXRkHbMCARz0aVfEt68P,35s58BRTGAEWztPo9WqCIs,2,2018-12-14,day
4,1rqqCSm0Qe4I9rUvWncaom,6ApYSpXF8GxZAgBTHDzYge,4,2018-06-22,day


In [37]:
song_features = acoustic_features.merge(lyrics, on = 'song_id')
song_features

,song_id,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,lyrics
0,3e9HZxeyfWwjeyPAMmWSSQ,207320,1,1,4,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,['[Verse 1]\nThought I\'d end up with Sean\nBu...
1,5p7ujcrUXASCNwRaWNHR1C,201661,6,1,4,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,"[""[Verse 1]\nFound you when your heart was bro..."
2,2xLMifQCjDGFmkHkpNLD9h,312820,8,1,4,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,"['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun..."
3,3KkXRkHbMCARz0aVfEt68P,158040,2,1,4,0.55600,0.760,0.479,0.000000,0.0703,-5.574,0.0466,0.913,89.911,NaN
4,1rqqCSm0Qe4I9rUvWncaom,190947,5,1,4,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,"[""[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20399,0yZsBLZVU2HTMlMqYvWevJ,133744,5,1,4,0.38700,0.453,0.647,0.000000,0.4510,-7.985,0.0277,0.728,71.357,['The Trade Winds\nMiscellaneous\nNew York\'s ...
20400,30342v7I9sUoNC0Djnu1mW,125293,1,1,4,0.64700,0.576,0.542,0.000000,0.1730,-8.371,0.0276,0.508,104.083,"[""There's nothing I can say\nThere's nothing I..."
20401,5rkgbGIOTBGlKTAawWb06X,139400,4,0,4,0.14400,0.300,0.743,0.000000,0.3850,-10.005,0.0393,0.952,154.180,['825 ALL NIGHT – PAROV STELAR\n824 BIG JET PL...
20402,2MVPsa4S4r7Ii5lMh85E03,131333,9,1,4,0.79100,0.488,0.830,0.000728,0.1550,-4.487,0.0402,0.950,181.869,NaN


In [39]:
df1 = song_features.merge(songs, on = 'song_id', suffixes=('', '_songs'))
len(df1)

20404

In [40]:
df2 = df1.merge(tracks, on = 'song_id', suffixes=('', '_tracks'))
len(df2)

20404

In [41]:
df3 = df2.merge(albums, on = 'album_id', suffixes=('', '_albums'))
len(df3)

11300

In [42]:
df4 = df3.merge(releases, on = 'album_id', suffixes=('', '_releases'))
len(df4)

11300

In [43]:
df5 = df4.merge(artists, on = 'artist_id', suffixes=('', '_artists'))
len(df5)

11296

In [44]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11296 entries, 0 to 11295
Data columns (total 42 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   song_id                          11296 non-null  object 
 1   duration_ms                      11296 non-null  int64  
 2   key                              11296 non-null  int64  
 3   mode                             11296 non-null  int64  
 4   time_signature                   11296 non-null  int64  
 5   acousticness                     11296 non-null  float64
 6   danceability                     11296 non-null  float64
 7   energy                           11296 non-null  float64
 8   instrumentalness                 11296 non-null  float64
 9   liveness                         11296 non-null  float64
 10  loudness                         11296 non-null  float64
 11  speechiness                      11296 non-null  float64
 12  valence           

In [45]:
df5.sample(10)

,song_id,duration_ms,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,...,artist_id,release_date_releases,release_date_precision_releases,name_artists,followers,popularity_artists,artist_type,main_genre,genres,image_url_artists
8139,2gO8oB0iRASgHstU5S6SLF,222325,6,1,4,0.101000,0.618,0.670,0.178000,0.0845,...,6Dpx8fRtNa9CEORHIiBQTG,1974-01-01,day,Lamont Dozier,11479.0,33,singer,classic soul,"['classic soul', 'motown', 'quiet storm', 'sou...",https://i.scdn.co/image/92b6628aa99a5feba22672...
3186,3tZm76otWH20xzJC7icHCk,270893,10,1,4,0.355000,0.856,0.755,0.000621,0.1450,...,2hWr3AjjKOCVmWcwvuT4uM,2001-10-16,day,Angie Stone,339255.0,59,singer,neo soul,"['neo soul', 'r&b', 'soul', 'urban contemporary']",https://i.scdn.co/image/5b128023e8c83a475685b6...
1070,5xhQChGGhKLWqBqX4XhtYE,259102,1,1,4,0.452000,0.825,0.435,0.609000,0.0953,...,4njdEjTnLfcGImKZu1iSrz,2013-11-19,day,AWOLNATION,1481796.0,69,band,la indie,"['la indie', 'modern alternative rock', 'moder...",https://i.scdn.co/image/3e110fc5cc102bb5b2ce0d...
4254,3mhOmh4tRKsMfnRmgZfeBm,299733,7,1,4,0.001870,0.552,0.645,0.000001,0.0424,...,3R6c2tWDBeYB1GncjLMn9n,1993-07-16,day,Candlebox,414118.0,59,band,alternative metal,"['alternative metal', 'alternative rock', 'gru...",https://i.scdn.co/image/d017775d0d78c93083e6f2...
9394,7a3g7jILDCvADVN9NQIyYj,239733,1,1,4,0.858000,0.635,0.167,0.000999,0.2130,...,09L3cUdx0hq6qn5bKuJJ4I,1968-05-01,day,Herb Alpert & The Tijuana Brass,64312.0,54,band,adult standards,"['adult standards', 'easy listening', 'exotica...",https://i.scdn.co/image/b7d355997334ec3352bcbf...
10817,5a4AQWs84xeEyYUd1iC2Pn,282093,7,1,4,0.004470,0.781,0.625,0.000000,0.1850,...,0VKTLKamj4IH8OfQbUL0kq,2006-01-01,day,Dem Franchize Boyz,187849.0,53,band,atl hip hop,"['atl hip hop', 'crunk', 'dirty south rap', 'g...",https://i.scdn.co/image/7f89ea1a94c3912412895a...
10465,3iJGrfXRepS04IiZP3VBUp,421043,10,0,4,0.000184,0.525,0.554,0.076500,0.0563,...,3X9xSaKYs3EY8l74Hgy5rz,2015-02-22,day,Crown Heights Affair,11113.0,32,band,deep disco,"['deep disco', 'disco', 'post-disco', 'rare gr...",https://i.scdn.co/image/6304f84c41b8395a5b6f17...
8726,1P8lIrb4yLlg1nzAjhhcfi,260843,11,0,4,0.404000,0.696,0.631,0.890000,0.1480,...,2moiE5ZUDX3Cni022mK9Rb,1971-11-06,day,Dennis Coffey & The Detroit Guitar Band,187.0,7,-,-,[],https://i.scdn.co/image/45c28e8c9db6b77edf2bfd...
8553,0pXm2TrQHRnw9BIpbRz9Nn,153973,1,1,4,0.472000,0.815,0.307,0.000319,0.0762,...,03PCdNvkF81s7xHmtkNcoz,1994-06-14,day,King Floyd,6829.0,48,singer,classic soul,"['classic soul', 'soul', 'southern soul']",https://i.scdn.co/image/ccb05e88be9fe889e6c1c9...
8168,0gXj72vNlNoh7kiC8zPEQp,201600,2,1,4,0.232000,0.528,0.421,0.000000,0.1590,...,6HX8AbXUFaYRtlqKb4CCo0,1974,year,Mac Davis,19737.0,42,singer,bubblegum pop,"['bubblegum pop', 'nashville sound', 'soft roc...",https://i.scdn.co/image/12f942c6eeca62dff58738...


In [46]:
df5.isnull().sum()

song_id                              0
duration_ms                          0
key                                  0
mode                                 0
time_signature                       0
acousticness                         0
danceability                         0
energy                               0
instrumentalness                     0
liveness                             0
loudness                             0
speechiness                          0
valence                              0
tempo                                0
lyrics                             256
song_name                            0
billboard                            0
artists                              0
popularity                           0
explicit                             0
song_type                            0
album_id                             0
track_number                         0
release_date                         0
release_date_precision               0
name                     